# Práctica de Aprendizaje Automático – Modelo Final

**Asignatura:** Aprendizaje Automático  
**Curso:** 2024–2025  
**Trabajo práctico grupal**  
**Alumnas:** Elena Recio Álvarez (100495725), Alejandra Castuera García (100499172)  
**Entrega:** Modelo final entrenado y predicciones para el conjunto de competición  

---

### ¿Qué contiene este notebook?

Este notebook recoge la **última fase del trabajo**, centrada en:

- Entrenar el **modelo final** con el conjunto completo de datos.
- Guardar ese modelo en un archivo (`modelo_final.pkl`) para su uso posterior.
- Realizar predicciones con ese modelo sobre los datos de competición (`competition_07.csv.gz`).
- Guardar dichas predicciones en el fichero `predicciones.csv`.

---

### Archivos utilizados

De acuerdo con las instrucciones del enunciado, los datos utilizados corresponden al grupo **07**, ya que se establece que `xx = a + b`, siendo `a` y `b` los dos últimos dígitos del NIA de uno de los miembros del grupo.

En nuestro caso, el NIA utilizado es **100495725**, por lo que:

- `a = 2`
- `b = 5`
- `2 + 5 = 7`

Por tanto, se han utilizado los siguientes ficheros:

- `attrition_availabledata_07.csv.gz`
- `attrition_competition_07.csv.gz`


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar dataset de entrenamiento
df = pd.read_csv("attrition_availabledata_07.csv.gz")

# Separar variables y target
X = df.drop(columns=["Attrition"])
y = df["Attrition"]

# Cargar dataset de competición (sin columna objetivo)
df_competition = pd.read_csv("attrition_competition_07.csv.gz")


## Entrenamiento del modelo final

Seleccionamos como mejor modelo la SVM con kernel RBF, tras ajuste de hiperparámetros (`C` y `gamma`) en el paso anterior.  
Entrenamos este modelo usando **todos los datos disponibles de entrenamiento**.


In [3]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Columnas numéricas y categóricas
columnas_num = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
columnas_cat = X.select_dtypes(include="object").columns.tolist()

# Preprocesado final
preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imp", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]), columnas_num),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]), columnas_cat)
])

# Modelo definitivo con hiperparámetros seleccionados
modelo_final = Pipeline([
    ("preprocessor", preprocessor),
    ("modelo", SVC(kernel="rbf", C=10, gamma=0.01, probability=True))
])

# Entrenar con TODO el set disponible
modelo_final.fit(X, y)

# Realizar predicciones sobre el conjunto de competición
y_pred = modelo_final.predict(df_competition)

# Crear el DataFrame de predicciones con EmployeeID original
predicciones = pd.DataFrame({
    "EmployeeID": df_competition["EmployeeID"],
    "Attrition": y_pred
})

# Guardar las predicciones en un archivo CSV
predicciones.to_csv("predicciones.csv", index=False)



In [4]:
import joblib

# Guardar el modelo entrenado
joblib.dump(modelo_final, "modelo_final.pkl")
print("Modelo final guardado como modelo_final.pkl")


Modelo final guardado como modelo_final.pkl


## Generar predicciones para el conjunto de competición

Cargamos el conjunto `attrition_competition_07.csv.gz`, aplicamos el modelo y guardamos las predicciones en un archivo `.csv` con formato de entrega.


In [5]:
# Predecir (etiquetas)
predicciones = modelo_final.predict(df_competition)

# Guardar en CSV con EmployeeID original
df_envio = pd.DataFrame({
    "EmployeeID": df_competition["EmployeeID"],
    "Attrition": predicciones
})
df_envio.to_csv("predicciones.csv", index=False)

print("Predicciones guardadas en predicciones.csv")



Predicciones guardadas en predicciones.csv


In [6]:
df_envio["Attrition"].value_counts()


,count
Attrition,
No,1343
Yes,127


## Conclusión

En este notebook se ha completado la fase final del proceso de modelado para predecir el abandono de empleados. Tras comparar varios modelos y técnicas, se seleccionó un **SVM con kernel RBF**, ajustado mediante búsqueda de hiperparámetros, como la mejor alternativa según el rendimiento obtenido en validación cruzada.

A partir de ahí, se procedió a:

- Entrenar el modelo final utilizando **todo el conjunto de entrenamiento** (`availabledata_07.csv.gz`).
- Guardar dicho modelo en un archivo (`modelo_final.pkl`) para su uso posterior.
- Aplicar el modelo sobre el **conjunto de competición** (`competition_07.csv.gz`) para generar predicciones.
- Exportar dichas predicciones en el archivo `predicciones.csv`, incluyendo el `EmployeeID` original y la etiqueta estimada (`Attrition`).

Además, se comprobó la **distribución de las predicciones**, con un total de:

- **1343 empleados clasificados como "No" (no abandonan)**
- **127 empleados clasificados como "Yes" (probabilidad de abandono)**

Esta distribución es coherente con el desequilibrio observado en el conjunto de entrenamiento, lo que indica que el modelo mantiene una interpretación razonable del problema.

Con este notebook, se completa todo el flujo de trabajo propuesto en la práctica: desde el análisis exploratorio hasta la construcción y entrega de un modelo final funcional y validado.


## Uso de IA

Durante la elaboración de esta parte del trabajo, se ha hecho un uso puntual de IA para resolver dudas concretas sobre el uso de funciones como `joblib.dump` o `predict()` y para comprobar que la estructura del código cumplía con lo exigido en el enunciado.

También se ha utilizado brevemente para revisar que el formato del archivo `predicciones.csv` fuera el adecuado.

Todo el código ha sido adaptado, revisado y entendido por el equipo antes de su inclusión en el notebook.
